# Trace Sentence Transfomer

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../../..')))

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="TracerWarning: torch.tensor")
warnings.filterwarnings("ignore", message="using SSL with verify_certs=False is insecure.")

import opensearch_py_ml as oml
from opensearchpy import OpenSearch
from opensearch_py_ml.ml_models import SentenceTransformerModel
# import mlcommon to later register the model to OpenSearch Cluster
from opensearch_py_ml.ml_commons import MLCommonClient

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# import torch
# from sentence_transformers import SentenceTransformer
# model_id = "sentence-transformers/msmarco-distilbert-base-tas-b"
# model = SentenceTransformer(model_id)
# device = torch.device("cpu")
# cpu_model = model.to(device)
# sentences = ['Sentence 1','Sentence 2']
# features = cpu_model.tokenizer(
#             sentences, return_tensors="pt", padding=True, truncation=True
#         ).to(device)
# ex_input = {
#                     "input_ids": features["input_ids"],
#                     "attention_mask": features["attention_mask"],
#                 }
# traced_model = torch.jit.trace(model, ex_input, strict=False)
# torch.jit.save(traced_model, "traced_sentence_transformer.pt")

In [4]:
CLUSTER_URL = 'https://localhost:9200'

In [5]:
def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client 

In [6]:
client = get_os_client()

# Connect to ml_common client with OpenSearch client
ml_client = MLCommonClient(client)

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [7]:
model_id="sentence-transformers/distiluse-base-multilingual-cased-v1"
pre_trained_model = SentenceTransformerModel(model_id=model_id, folder_path='sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1', overwrite=True)
model_path = pre_trained_model.save_as_pt(model_id=model_id, sentences=["for example providing a small sentence", "we can add multiple sentences"])

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


model file is saved to  sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/distiluse-base-multilingual-cased-v1.pt
zip file is saved to  sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/distiluse-base-multilingual-cased-v1.zip 



# Load Sentence Transformer

In [10]:
import torch
loaded_model = torch.jit.load("sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/distiluse-base-multilingual-cased-v1.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=SentenceTransformer
  (0): RecursiveScriptModule(
    original_name=Transformer
    (auto_model): RecursiveScriptModule(
      original_name=DistilBertModel
      (embeddings): RecursiveScriptModule(
        original_name=Embeddings
        (word_embeddings): RecursiveScriptModule(original_name=Embedding)
        (position_embeddings): RecursiveScriptModule(original_name=Embedding)
        (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
        (dropout): RecursiveScriptModule(original_name=Dropout)
      )
      (transformer): RecursiveScriptModule(
        original_name=Transformer
        (layer): RecursiveScriptModule(
          original_name=ModuleList
          (0): RecursiveScriptModule(
            original_name=TransformerBlock
            (attention): RecursiveScriptModule(
              original_name=MultiHeadSelfAttention
              (dropout): RecursiveScriptModule(original_name=Dropout)
              (q_lin): RecursiveS

In [59]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_id)
device = torch.device("cpu")
cpu_model = model.to(device)

test_sentences = ["first sentence", "second much longer sentence"]
features = cpu_model.tokenizer(
            test_sentences, return_tensors="pt", padding=True, truncation=True
        ).to(device)
test_input = {
                    "input_ids": features["input_ids"],
                    "attention_mask": features["attention_mask"],
                }

pt_embedding = loaded_model(test_input)
pt_embedding

{'input_ids': tensor([[  101, 10422, 49219,   102,     0,     0],
         [  101, 11132, 13172, 20165, 49219,   102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1]]),
 'token_embeddings': tensor([[[-0.0932,  0.0432,  0.1432,  ..., -0.1103,  0.0942, -0.0037],
          [-0.1012,  0.0241,  0.1627,  ..., -0.0762,  0.0517,  0.0244],
          [-0.1129,  0.1023,  0.0799,  ..., -0.0908,  0.1273, -0.0619],
          [-0.0190, -0.0165,  0.0428,  ..., -0.0710, -0.0092,  0.0306],
          [-0.0025,  0.0387,  0.0554,  ..., -0.0957,  0.1055,  0.0341],
          [-0.0489,  0.0154,  0.0672,  ..., -0.1079,  0.1109, -0.0167]],
 
         [[-0.1305,  0.0559,  0.0766,  ..., -0.0346,  0.0872,  0.0860],
          [-0.1149,  0.0713,  0.0938,  ..., -0.0121,  0.0341,  0.0632],
          [-0.0466,  0.0071,  0.0116,  ..., -0.0323,  0.0609,  0.1089],
          [-0.0728,  0.0599,  0.0552,  ..., -0.0520,  0.0804,  0.0945],
          [-0.1406,  0.1172, -0.0054,  ..., -0.0232,  0.08

In [60]:
pt_embedding['sentence_embedding'].detach().cpu().numpy().shape

(2, 512)

# Compare Embedding

In [61]:
model = SentenceTransformer(model_id)

In [62]:
original_embedding = model.encode(test_sentences, convert_to_numpy=True)
original_embedding

array([[ 0.01097566,  0.06483249, -0.04571173, ..., -0.04982213,
         0.01845153, -0.03800954],
       [ 0.04462758,  0.05222933,  0.0042971 , ..., -0.07785919,
         0.06257077, -0.00566589]], dtype=float32)

In [63]:
import numpy as np
print(np.testing.assert_allclose(original_embedding, pt_embedding['sentence_embedding'].detach().cpu().numpy(), rtol=1e-03, atol=1e-05))

None


In [64]:
features

{'input_ids': tensor([[  101, 10422, 49219,   102,     0,     0],
        [  101, 11132, 13172, 20165, 49219,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}

In [65]:
from transformers import AutoTokenizer

In [66]:
autotokenizer = AutoTokenizer.from_pretrained("sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/")

In [67]:
auto_features = autotokenizer(
            test_sentences, return_tensors="pt", padding=True, truncation=True
        )

In [68]:
auto_features 

{'input_ids': tensor([[  101, 10422, 49219,   102,     0,     0],
        [  101, 11132, 13172, 20165, 49219,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}

In [69]:
import zipfile

fantasy_zip = zipfile.ZipFile("sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/distiluse-base-multilingual-cased-v1.zip")
fantasy_zip.extractall("sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/extract")

fantasy_zip.close()

In [70]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/extract/tokenizer.json")

In [71]:
fast_tokenizer.add_special_tokens({
  "cls_token": "[CLS]",
  "mask_token": "[MASK]",
  "pad_token": "[PAD]",
  "sep_token": "[SEP]",
  "unk_token": "[UNK]"
})

0

In [72]:
fast_tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=119547, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [73]:
fast_tokenizer.save_pretrained("sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/new")

('sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/new/tokenizer_config.json',
 'sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/new/special_tokens_map.json',
 'sentence-transformers-torchscript/distiluse-base-multilingual-cased-v1/new/tokenizer.json')

In [74]:
fast_features = fast_tokenizer(
            test_sentences, return_tensors="pt", padding=True, truncation=True
        )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [75]:
fast_features

{'input_ids': tensor([[  101, 10422, 49219,   102,     0,     0],
        [  101, 11132, 13172, 20165, 49219,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}